# Optimal weight table

In [2]:
import numpy as np
import networkx as nx
from itertools import product
import random
import matplotlib.pyplot as plt

In [3]:
length = 3
alphabet = ['A', 'C', 'G', 'U']
nodes = [''.join(p) for p in product(alphabet, repeat=length)]

In [4]:
SGC = [
    ["UUU", "UUC"],  # Phenylalanine (Phe)
    ["UUA", "UUG", "CUU", "CUC", "CUA", "CUG"],  # Leucine (Leu)
    ["AUU", "AUC", "AUA"],  # Isoleucine (Ile)
    ["AUG"],  # Methionine (Met) - Start codon
    ["GUU", "GUC", "GUA", "GUG"],  # Valine (Val)
    ["UCU", "UCC", "UCA", "UCG", "AGU", "AGC"],  # Serine (Ser)
    ["CCU", "CCC", "CCA", "CCG"],  # Proline (Pro)
    ["ACU", "ACC", "ACA", "ACG"],  # Threonine (Thr)
    ["GCU", "GCC", "GCA", "GCG"],  # Alanine (Ala)
    ["UAU", "UAC"],  # Tyrosine (Tyr)
    ["UAA", "UAG", "UGA"],  # Stop codons
    ["CAU", "CAC"],  # Histidine (His)
    ["CAA", "CAG"],  # Glutamine (Gln)
    ["AAU", "AAC"],  # Asparagine (Asn)
    ["AAA", "AAG"],  # Lysine (Lys)
    ["GAU", "GAC"],  # Aspartic acid (Asp)
    ["GAA", "GAG"],  # Glutamic acid (Glu)
    ["UGU", "UGC"],  # Cysteine (Cys)
    ["UGG"],  # Tryptophan (Trp)
    ["CGU", "CGC", "CGA", "CGG", "AGA", "AGG"],  # Arginine (Arg)
    ["GGU", "GGC", "GGA", "GGG"],  # Glycine (Gly)
]

In [62]:
swap_index =  {
    0: {#A
        0: 
        #C    #G    #U   
        {1: 0, 2: 1, 3: 2},
        
        #C
        1:
        #G    #U
        {2: 3, 3: 4},
        
        #G
        2:
        #U
        {3: 5}
        },
    1: {#A
        0: 
        #C    #G    #U   
        {1: 6, 2: 7, 3: 8},
        
        #C
        1:
        #G    #U
        {2: 9, 3: 10},
        
        #G
        2:
        #U
        {3: 11}
        },
    2: {#A
        0: 
        #C    #G    #U   
        {1: 12, 2: 13, 3: 14},
        
        #C
        1:
        #G    #U
        {2: 15, 3: 16},
        
        #G
        2:
        #U
        {3: 17}
        },
}

table_ones = np.ones(18)

table_altered = [1, 1, 1, 1, 1, 1,
                 1, 1, 1, 1, 1, 1,
                 2, 4, 2, 2, 4, 2]

In [63]:
def weights(s1, s2, table):
    if sum(a != b for a, b in zip(s1, s2)) == 1:
        if s1[0] != s2[0]:
            # base 1
            letter1 = s1[0]
            letter2 = s2[0]
            base = 0
        elif s1[1] != s2[1]:
            # base 2
            letter1 = s1[1]
            letter2 = s2[1]
            base = 1
        else:
            # base3
            letter1 = s1[2]
            letter2 = s2[2]
            base = 2
    
        l1 = ['A', 'C', 'G', 'U'].index(letter1)
        l2 = ['A', 'C', 'G', 'U'].index(letter2)
        if l1 > l2:
            l1, l2 = l2, l1
        return table[swap_index[base][l1][l2]]

In [64]:
def conductance(S, G):
    numerator = 0
    denominator = 0
    for edge, w in nx.get_edge_attributes(G, "weight").items():
        node1, node2 = edge
        if node1 in S and node2 in S:
            denominator += 2 * w
        else:
            numerator += w
            denominator += w

    return numerator / denominator

In [69]:
def mean_conductance(table):
    conductances = []
    for S in SGC:
        G = nx.Graph()
        for i, node1 in enumerate(S):
            for node2 in nodes:
                if node2 == node1:
                    continue
                w = weights(node1, node2, table)
                if w:
                    G.add_edge(node1, node2, weight=w)
                    
        conductances.append(conductance(S, G))

    return np.mean(conductances)

In [70]:
def fitness(table):
    return 1 - mean_conductance(table)

In [72]:
for table in table_ones, table_altered:
    print(fitness(table))

0.1887125220458553
0.3605442176870749


In [146]:
bias_factor = 1
pop_size = 10
probabilities = np.exp(-bias_factor * np.arange(pop_size) / pop_size)
probabilities /= np.sum(probabilities)
print(probabilities)
print(np.random.choice(pop_size, size=2, p=probabilities, replace=False))

[0.15054499 0.13621874 0.12325581 0.11152647 0.10091332 0.09131015
 0.08262084 0.07475843 0.06764422 0.06120702]
[8 5]


In [ ]:
def initialize_population(pop_size, num_weights):
    return np.random.rand(pop_size, num_weights)

def crossover(parent1, parent2):
    crossover_point = np.random.randint(1, 18)
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
    return child1, child2

def mutate(weights, mutation_rate=0.4):
    index = np.random.randint(18)
    factor = np.random.uniform(1 / (1 + mutation_rate), (1 + mutation_rate))
    weights[index] *= factor
    return weights

def evolutionary_algorithm(fitness, pop_size=150, crossover_rate=0.6, mutation_rate=0.4, delta=1e-6, max_generations=1000, bias_factor=1):
    population = initialize_population(pop_size, 18)
    best_fitness = 0
    generations = 0
    
    probabilities = np.exp(-bias_factor * np.arange(pop_size) / pop_size)
    probabilities /= np.sum(probabilities)
    
    while generations < max_generations:
        fitness_values = np.array([fitness(ind) for ind in population])
        sorted_indices = np.argsort(fitness_values)[::-1]
        population = population[sorted_indices]
        new_population = []
      
        for i in range(int(crossover_rate * pop_size / 2)):
            parent1_index, parent2_index = np.random.choice(pop_size, size=2, p=probabilities, replace=False)
            parent1, parent2 = population[parent1_index], population[parent2_index]
            child1, child2 = crossover(parent1, parent2)
            new_population.extend([child1, child2])
        
        while len(new_population) < pop_size:
            new_population.append(population[np.random.choice(pop_size, p=probabilities)].copy())
        
        new_population = np.array([mutate(ind, mutation_rate) for ind in new_population])
        
        new_best_fitness = max(fitness_values)
        if new_best_fitness > best_fitness:
            best_fitness = new_best_fitness
        print(f"{best_fitness:.5f}, {new_best_fitness:.5f}", generations)
        
        population = new_population
        generations += 1
    
    fitness_values = np.array([fitness(ind) for ind in population])
    sorted_indices = np.argsort(fitness_values)[::-1]
    population = population[sorted_indices]
    best_individual = population[0]
    
    return best_individual / np.sum(best_individual) * 18  # Normalize weights

In [209]:
best_ind = evolutionary_algorithm(fitness, pop_size=50, max_generations=500)

0.28465, 0.28465 0
0.29473, 0.29473 1
0.29473, 0.28142 2
0.29473, 0.29174 3
0.29473, 0.28927 4
0.29473, 0.28496 5
0.31514, 0.31514 6
0.32992, 0.32992 7
0.36669, 0.36669 8
0.37117, 0.37117 9
0.37117, 0.37011 10
0.37117, 0.36172 11
0.37117, 0.36186 12
0.37117, 0.36681 13
0.37336, 0.37336 14
0.37780, 0.37780 15
0.38011, 0.38011 16
0.39454, 0.39454 17
0.41124, 0.41124 18
0.41124, 0.39330 19
0.41124, 0.39468 20
0.42997, 0.42997 21
0.42997, 0.42942 22
0.43278, 0.43278 23
0.43278, 0.43197 24
0.43278, 0.42821 25
0.43278, 0.43140 26
0.44264, 0.44264 27
0.45816, 0.45816 28
0.46074, 0.46074 29
0.46409, 0.46409 30
0.49664, 0.49664 31
0.50159, 0.50159 32
0.50159, 0.50150 33
0.51125, 0.51125 34
0.51125, 0.51071 35
0.51125, 0.49816 36
0.51125, 0.50663 37
0.51125, 0.50380 38
0.53335, 0.53335 39
0.53340, 0.53340 40
0.55204, 0.55204 41
0.55536, 0.55536 42
0.55536, 0.55490 43
0.57512, 0.57512 44
0.58361, 0.58361 45
0.58763, 0.58763 46
0.60180, 0.60180 47
0.60243, 0.60243 48
0.60243, 0.60228 49
0.62009, 0

In [221]:
np.set_printoptions(precision=10, suppress=True)
print(best_ind)

print(mean_conductance(best_ind))
print(fitness(best_ind))

[ 0.0000000312  0.0000000131  0.0000000197  0.0000000144  0.000000008
  0.0000000354  0.0000000027  0.0000000214  0.0000000174  0.0000001933
  0.0000003351  0.0000000895  0.0000000004  0.0052936225  0.0000000702
  0.0000000596 17.9947053891  0.000000077 ]
0.1111252633105997
0.8888747366894003


In [216]:
print(mean_conductance([1, 1, 1, 1, 1, 1,
                        1, 1, 1, 1, 1, 1,
                        1, 1, 1, 1, 10, 1]))

0.5865961199294534


In [217]:
print(mean_conductance([1, 1, 1, 1, 1, 1,
                        1, 1, 1, 1, 1, 1,
                        1, 1, 1, 1, 100, 1]))

0.3306533325580945


In [218]:
print(mean_conductance([1, 1, 1, 1, 1, 1,
                        1, 1, 1, 1, 1, 1,
                        1, 1, 1, 1, 1000, 1]))

0.27049853567445475


In [219]:
print(mean_conductance([1, 1, 1, 1, 1, 1,
                        1, 1, 1, 1, 1, 1,
                        1, 1, 1, 1, 10000, 1]))

0.2635686954224119


In [220]:
print(mean_conductance([1, 1, 1, 1, 1, 1,
                        1, 1, 1, 1, 1, 1,
                        1, 1, 1, 1, 100000, 1]))

0.26286491709245047


In [222]:
print(mean_conductance([1, 1, 1, 1, 1, 1,
                        1, 1, 1, 1, 1, 1,
                        1, 10, 1, 1, 100000, 1]))

0.18702931098792552


In [ ]:
print(mean_conductance([1, 1, 1, 1, 1, 1,
                        1, 1, 1, 1, 1, 1,
                        1, 100, 1, 1, 100000, 1]))

0.11779950724278339


In [233]:
print(mean_conductance([1, 1, 1, 1, 1, 1,
                        1, 1, 1, 1, 1, 1,
                        1, 1000, 1, 1, 100000, 1]))

0.11277965635521481


In [234]:
print(mean_conductance([1, 1, 1, 1, 1, 1,
                        1, 1, 1, 1, 1, 1,
                        1, 10000, 1, 1, 100000, 1]))

0.11358700218197197
